# Build community model

In [1]:
from __future__ import print_function, division, absolute_import

import sys

import qminospy
from qminospy.me2 import ME_NLP

# python imports
from copy import copy
import re
from os.path import join, dirname, abspath
import sys
from collections import defaultdict
import pickle

# third party imports
import pandas
import cobra
from tqdm import tqdm
import numpy as np
import scipy

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import bacillusme
from bacillusme import (transcription, translation, flat_files, generics, formulas, compartments)
from bacillusme.util.helper_functions import *

import copy
%load_ext autoreload
%autoreload 2
print(cobra.__file__)
print(cobrame.__file__)
print(bacillusme.__file__)
ecoli_files = dirname(abspath(bacillusme.__file__))
plt.rcParams['svg.fonttype'] = 'none'

/home/jt/work/cobrapy-0.5.11/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/home/jt/work/cobrapy-0.5.11/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


/home/jt/work/cobrapy-0.5.11/cobra/__init__.py
/home/jt/work/cobrame/cobrame/__init__.py
/home/jt/work/bacillusme-master/bacillusme/__init__.py


In [2]:
eco_directory = join(flat_files.ecoli_files_dir, 'iJO1366.json')
ijo_directory = join(flat_files.ecoli_files_dir, 'iYO844.json')
uni_directory = join(flat_files.ecoli_files_dir, 'universal_model.json')

eco = cobra.io.load_json_model(eco_directory)
bsub = cobra.io.load_json_model(ijo_directory)
uni = cobra.io.load_json_model(uni_directory)

bsub.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16


<Solution 0.12 at 0x7fc605eb1590>

In [3]:
with open(ecoli_files+'/me_models/iJT964-ME.pickle', 'rb') as solution:
    me = pickle.load(solution)
with open(ecoli_files+'/me_models/iJT964-ME.pickle', 'rb') as solution:
    spore = pickle.load(solution)

## Create spore model

### Change IDs (compartment _s)

In [4]:
# Metabolites
new_compartment = 's'
fix = []
for met in tqdm(spore.metabolites):
    if re.search('_[c,e]$',met.id):
        met.id = re.sub('c$',new_compartment,met.id)
    else:
        met.id = met.id+'_'+new_compartment

100%|██████████| 4211/4211 [00:00<00:00, 837627.53it/s]


In [5]:
# Reactions
new_compartment = 's'
fix = []
for r in tqdm(spore.reactions):
    r.id = r.id+'_'+new_compartment

100%|██████████| 6277/6277 [00:00<00:00, 1707037.94it/s]


In [6]:
spore.repair()

In [7]:
# Create new growth key for spore
new_growth_key = 'sigma'
for r in tqdm(spore.reactions):
    lb = r.lower_bound
    ub = r.upper_bound
    new_stoichiometry = {m:s for m,s in r.metabolites.items()}
    if hasattr(lb, 'subs'):
        growth_key = list(lb.free_symbols)[0]
        r.lower_bound = lb.subs(growth_key,new_growth_key)
    if hasattr(ub, 'subs'):
        growth_key = list(ub.free_symbols)[0]
        r.upper_bound = ub.subs(growth_key,new_growth_key)
    for m,s in new_stoichiometry.items():
        if hasattr(s, 'subs'):
            growth_key = list(s.free_symbols)[0]
            new_stoichiometry[m] = s.subs(growth_key,new_growth_key)
    r.add_metabolites(new_stoichiometry,combine=False)

100%|██████████| 6277/6277 [00:08<00:00, 779.66it/s] 


In [8]:
# Create new growth key for spore
new_growth_key = 'sigma'
for r in tqdm(spore.reactions):
    

SyntaxError: unexpected EOF while parsing (848353231.py, line 4)

In [ ]:
for r in get_all_transport_of_model(spore):
    rxn = spore.reactions.get_by_id(r)
    if not isinstance(rxn,cobrame.core.reaction.MetabolicReaction): continue
    print(rxn.id)
    break

In [ ]:
# Eliminate exchange reactions:
reactions = [r for r in spore.reactions]
for r in tqdm(reactions):
    if re.search('^EX_',r.id):
        r.remove_from_model()

### Create spore biomass reactions

In [ ]:
bio_rxn = spore.reactions.BIOMASS_BS_10_FWD_CPLX_dummy_s

In [ ]:
bio_comp = pd.read_csv('spore_composition.csv',index_col=0)

In [ ]:
# Identify lipid metabolites in biomass equation
lipid_demand = {}
for m in bio_comp.index:
    if '_BS_c' in m:
        new_id = re.sub('c$','s',m)
        value = bio_comp.loc[m]['coeff']
        lipid_demand[new_id] = abs(value)
                
for met, requirement in lipid_demand.items():
    component_mass = spore.metabolites.get_by_id(met).formula_weight / 1000.
    rxn = cobrame.SummaryVariable('Demand_' + met)
    spore.add_reactions([rxn])
    rxn.add_metabolites({met: -1 * requirement,
                         'lipid_biomass_s': component_mass * requirement})
    rxn.lower_bound = spore.reactions.biomass_dilution_s.lower_bound
    rxn.upper_bound = 1000.
#     print(rxn.reaction)

# Menaquinon is required in the MC for energy and electron transport https://www.ncbi.nlm.nih.gov/pmc/articles/PMC246562/.
spore.reactions.biomass_constituent_demand_s.add_metabolites({
    spore.metabolites.mql7_s:-spore.reactions.biomass_constituent_demand_s.metabolites[spore.metabolites.mql7_s]
})

### Merge

In [ ]:
with open(ecoli_files+'/me_models/solution.pickle', 'rb') as solution:
    sporeme = pickle.load(solution)

In [ ]:
# Mother is not growing. Reactions should then be associated to sigma
new_growth_key = 'sigma'
for r in tqdm(sporeme.reactions):
    lb = r.lower_bound
    ub = r.upper_bound
    new_stoichiometry = {m:s for m,s in r.metabolites.items()}
    if hasattr(lb, 'subs'):
        growth_key = list(lb.free_symbols)[0]
        r.lower_bound = float(lb.subs(growth_key,0.))
    if hasattr(ub, 'subs'):
        growth_key = list(ub.free_symbols)[0]
        r.upper_bound = float(ub.subs(growth_key,1000.))
    for m,s in new_stoichiometry.items():
        if hasattr(s, 'subs'):
            growth_key = list(s.free_symbols)[0]
            new_stoichiometry[m] = s.subs(growth_key,new_growth_key)
            a = 1
    r.add_metabolites(new_stoichiometry,combine=False)

In [ ]:
# sporeme.add_reactions(spore.reactions) # This approach no longer worked after python version change.

In [ ]:
# Add spore reactions
for r in spore.reactions:
    new = r.__class__(r.id) # Create and identical reaction
    new.bounds = r.bounds
    sporeme.add_reaction(new)
    for k,v in r.metabolites.items():
        if not hasattr(sporeme.metabolites,k.id):
            args = k.__init__.__code__.co_varnames[1:] # Eliminate "self"
            if len(args) == 1: # No additional inputs to __init__
                met = k.__class__(k.id)
            else:
                vals = [getattr(k, dir(k)[[i.lower() for i in dir(k)].index(a)]) for a in args]
                met = k.__class__(*vals)
            sporeme.add_metabolites(met)
            met.name = k.name
            met.formula = k.formula
            met.charge = k.charge
            met.compartment = 's'
            
        new.add_metabolites({k.id:v})

In [ ]:
# Connect C with S
## Change spore transport from e-s to c-s
for m in tqdm(sporeme.metabolites):
    if not isinstance(m,cobrame.Metabolite) or not re.search('_s$',m.id):
        continue
    in_transport = get_transport_reactions(sporeme,m.id,comps=['e','s'])
    out_transport = get_transport_reactions(sporeme,m.id,comps=['s','e'])
    transport_reactions = in_transport + out_transport
    for r in transport_reactions:
        old_met = sporeme.metabolites.get_by_id(re.sub('_s$','_e',m.id))
        new_met = sporeme.metabolites.get_by_id(re.sub('_s$','_c',m.id))
        coeff = r.metabolites[old_met]
        r.add_metabolites({old_met:-coeff,new_met:coeff})

In [ ]:
sporeme.objective = 'biomass_dilution_s'
sporeme.reactions.get_by_id('BIOMASS_BS_10_FWD_CPLX_dummy_s').upper_bound = 0

### Manual fixes

As reported by Eammon, arginine is transported through permeases RocC and RocE, and ABC transporters ArtPQR.

In [ ]:
# close_met_transport = ['3gmp_c','gmp_c']
# transporters = []
# for m in tqdm(close_met_transport):
#     transport = get_transport_reactions(sporeme,m,comps=['c','s']) + \
#         get_transport_reactions(sporeme,m,comps=['s','c'])
#     [transporters.append(i) for i in transport]
# transporters

In [ ]:
# for t in transporters:
#     t.lower_bound = 0
#     t.upper_bound = 0

### Close specific reactions in the spore
It was shown that all but one (odhB) TCA reaction was necessary. We close the unnecessary ones.

In [ ]:
# From sigE-regulated genes in BsubCyc
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4380625/
gene_dictionary = pd.read_csv('../../building_data/gene_name_dictionary.csv',index_col=1)

sigE_genes = pd.read_csv('./sigE_genes.txt',sep='\t')['Regulation - genes directly activated by gene'][0].split(' // ')
# sigE_genes = set(sigE_genes) - set(['BSU26390','BSU25760','BSU25070'])
sigE_genes = set(sigE_genes) - set(['BSU26390','BSU25760'])

sigE_genes = set()

gene_dictionary.reset_index().set_index('locus_id').loc[sigE_genes].reset_index().set_index('name')

In [ ]:
# From proteomics. ilvE = ybgE
# sigE_genes = set()
deplete_genes = set(['pckA','purL','ilvB','ilvE','citZ','acsA','etfA','acoC','sigF','yxbC','lutC',
                  'argJ','hpf'])
# eliminate_genes += ['sigE']

deplete_genes = set(gene_dictionary.loc[deplete_genes]['locus_id'])
deplete_genes

In [ ]:
eliminate_genes = deplete_genes | sigE_genes

In [ ]:
# # From Eammon's paper.
# # https://www.science.org/doi/10.1126/sciadv.abd6385
# eliminate_genes = ['citB','citZ','icd','sucD','fumC','mdh']
# gene_dictionary.loc[eliminate_genes]['locus_id']

In [ ]:
essentiality_df = pd.read_csv('../essentiality/essentiality_results.csv',index_col=0)
eliminate_ids = set(eliminate_genes) & set(essentiality_df.index)
essentiality_df.loc[eliminate_ids]

In [ ]:
for gene in eliminate_ids:
    r = 'translation_' + gene +'_s'
    print(r)
    rxn = sporeme.reactions.get_by_id(r)
    rxn.bounds = (0,0)

In [ ]:
# Close ATP synthase as it seems it is not present in the spore according to Eammons thesis
sporeme.reactions.get_by_id('ATPS4r_FWD_CPLX000-10_s').bounds = (0,0)
# No glucose uptake from e
sporeme.reactions.get_by_id('GLCpts_FWD_BSU13890-MONOMER_s').bounds = (0,0)

In [ ]:
for m in sporeme.metabolites:
    for r in m.reactions:
        r.id

### Solve

In [ ]:
# def add_transport_to_model(model,mets):
#     for met in mets:
#         rxn = cobrame.MEReaction('{}_transport'.format(met))
#         model.add_reactions([rxn])
#         rxn.add_metabolites({
#             '{}_c'.format(met):-1,
#             '{}_s'.format(met):1
#         })
#         rxn.bounds = (-1000,1000)
#         print(rxn.reaction)
# add_transport_to_model(sporeme,['ctp','gtp'])

In [ ]:
# sporeme.reactions.atp_transport.bounds = (0,0)

In [ ]:
# sporeme.reactions.translation_BSU26390_s.bounds = (0,1000)
# sporeme.reactions.translation_BSU25760_s.bounds = (0,1000)

In [ ]:
solve_me_model(sporeme, max_mu = 0.12, min_mu = .05, using_soplex=False, precision = 1e-6,growth_key = 'sigma')

In [ ]:
sporeme.solution

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
flux_based_reactions(sporeme,'arg__L_c',growth_symbol='sigma').head(5)

In [ ]:
df = flux_based_reactions(sporeme,'atp_s',growth_symbol='sigma',only_types=['MetabolicReaction'])

In [ ]:
df.head(20)

In [ ]:
sporeme.reactions.get_by_id('PYK_FWD_BSU29180-MONOMER_mod_mn2_mod_k_s')

In [ ]:
flux_based_reactions(sporeme,'pep_s',growth_symbol='sigma',only_types=['MetabolicReaction'])

In [ ]:
sporeme.metabolites.pep_s.reactions

In [ ]:
flux_based_reactions(sporeme,'adp_s',growth_symbol='sigma').head(5)

In [ ]:
flux_based_reactions(sporeme,'CPLX8J2-60',growth_symbol='sigma')

In [ ]:
['BSU26390','BSU25760','BSU25070']

In [ ]:
sporeme.name = 'Spore model with Eammons proteomics constraints'
with open("./sporeme_solution_v4_proteomics.pickle", "wb") as outfile:
    pickle.dump(sporeme, outfile)

In [ ]:
with open("./sporeme_solution_v4_proteomics.pickle", "rb") as outfile:
    sporeme = pickle.load(outfile)

In [ ]:
len(sporeme.metabolites),len(sporeme.reactions),len(sporeme.metabolites)

In [ ]:
len(me.metabolites),len(me.reactions),len([m for m in me.metabolites if 'RNA_' in m.id])

In [ ]:
from cobrame.util.helper_functions import flux_based_reactions

In [ ]:
get_reactions_of_met(sporeme,'cdp_s', s = 1)

In [ ]:
get_reactions_of_met(sporeme,'BSU37150-MONOMER_mod_mg2')

In [ ]:
for r in sporeme.reactions.query('CTPS'):
    print(r.id,r.reaction)

In [ ]:
print(sporeme.solution.f)

In [ ]:
df = pd.read_csv('PEP_essential_metabolite_group_KO.csv',index_col=0)

In [ ]:
for r,row in df.iterrows():
    if r not in sporeme.reactions:
        print(r)

In [ ]:
for r in sporeme.reactions:
    print(r.id)

In [ ]:
len(sporeme.reactions)

In [ ]:
len(me.reactions) + len(spore.reactions)

In [ ]:
sporeme.reactions.get_by_id('transcription_TU8J2_1243_from_BSU25200-MONOMER_s').id

In [ ]:
with open("./sporeme_solution_v2.pickle", "rb") as outfile:
    sporeme = pickle.load(outfile)

In [ ]:
get_reactions_of_met(sporeme,'malcoa_c',only_types=['MetabolicReaction'])

In [ ]:
len([r for r in sporeme.reactions if '_s' not in r.id[-2:]])

In [ ]:
len([r for r in sporeme.metabolites if '_s' in r.id[-2:]])

In [ ]:
len(sporeme.reactions) - 6282

In [ ]:
get_breakdown(sporeme,'metabolites').sort_values('count').plot.bar()
plt.savefig('figures/sporeme_metabolites.svg',format='SVG')

In [ ]:
get_breakdown(sporeme,'reactions').sort_values('count').plot.bar()
plt.savefig('figures/sporeme_reactions.svg',format='SVG')